# Data Processing for the Data Instance

In [1]:
# Importing packages
import pandas as pd
import numpy as np
import pickle

In [2]:
df_gen_raw = pd.read_csv('Data/ieee24rts_generators.csv', index_col=False)
df_line_raw = pd.read_csv('Data/ieee24rts_lines.csv', index_col=False)
df_load_raw = pd.read_csv('Data/ieee24rts_loads.csv', index_col=False)
df_aggr_raw = pd.read_csv('Data/ieee24rts_aggrs.csv', index_col=False)

## Generators

In [3]:
# Bus Index Set
B = set()
for i in range(1,25): # 24 bus system
    B.add(i)
# Generator Index Set
G = set()
for i in df_gen_raw.index:
    gen = (df_gen_raw.loc[i, 'Bus'], df_gen_raw.loc[i,'Generator'])
    G.add(gen)

In [4]:
# Generating dictionaries for generator parameters
ag_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='a')
ag = ag_matrix.stack().to_dict()

bg_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='b')
bg = bg_matrix.stack().to_dict()

cg_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='c')
cg = cg_matrix.stack().to_dict()

pgmax_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='Pmax')
p_g_max = pgmax_matrix.stack().to_dict()

pgmin_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='Pmin')
p_g_min = pgmin_matrix.stack().to_dict()

qgmax_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='Qmax')
q_g_max = qgmax_matrix.stack().to_dict()

qgmin_matrix = df_gen_raw.pivot_table(index='Bus', columns='Generator', values='Qmin')
q_g_min = qgmin_matrix.stack().to_dict()

In [5]:
# Exporting generator set and dictionaries as .pkl 
with open('Data/Parameters/B.pkl', 'wb') as f:
    pickle.dump(B, f)

with open('Data/Parameters/G.pkl', 'wb') as f:
    pickle.dump(G, f)

with open('Data/Parameters/ag.pkl', 'wb') as f:
    pickle.dump(ag, f)
    
with open('Data/Parameters/bg.pkl', 'wb') as f:
    pickle.dump(bg, f)
    
with open('Data/Parameters/cg.pkl', 'wb') as f:
    pickle.dump(cg, f)
    
with open('Data/Parameters/p_g_max.pkl', 'wb') as f:
    pickle.dump(p_g_max, f)
    
with open('Data/Parameters/p_g_min.pkl', 'wb') as f:
    pickle.dump(p_g_min, f)
    
with open('Data/Parameters/q_g_max.pkl', 'wb') as f:
    pickle.dump(q_g_max, f)
    
with open('Data/Parameters/q_g_min.pkl', 'wb') as f:
    pickle.dump(q_g_min, f)

## Lines

In [6]:
# Line Index Set
L = []
for i in df_line_raw.index:
    line = (df_line_raw.loc[i, 'From'], df_line_raw.loc[i, 'To'])
    L.append(line)

# Combining double-circuit lines for analyses
double_lines = [] 
for line in L:
    if L.count(line) != 1:
        double_lines.append(line)
double_lines = list(set(double_lines))
L = set(L)

In [7]:
cplx = 1j # Sqrt of 1
admittance = np.divide(1, (df_line_raw['r'] + df_line_raw['x']*cplx))
df_line_raw['gg'] = np.real(admittance)
df_line_raw['bb'] = np.imag(admittance)

In [8]:
# Combining double-circuit lines to single lines
for i in df_line_raw.index:
    for double_line in double_lines:
        if (double_line[0] == df_line_raw.loc[i, 'From']) and (double_line[1] == df_line_raw.loc[i, 'To']):
            df_line_raw.loc[i, 'r'] = 0.5*df_line_raw.loc[i, 'r']
            df_line_raw.loc[i, 'x'] = 0.5*df_line_raw.loc[i, 'x']
            df_line_raw.loc[i, 'Smax'] = 2*df_line_raw.loc[i, 'Smax']
            df_line_raw.loc[i, 'gg'] = 2*df_line_raw.loc[i, 'gg']
            df_line_raw.loc[i, 'bb'] = 2*df_line_raw.loc[i, 'bb']

df_line_raw = df_line_raw.drop_duplicates() # Dropping duplicate rows after modifying double-circuits into single-circuits

In [9]:
# Generating dictionaries for line parameters
gg_matrix = df_line_raw.pivot_table(index='From', columns='To', values='gg')
gg = gg_matrix.stack().to_dict()

bb_matrix = df_line_raw.pivot_table(index='From', columns='To', values='bb')
bb = bb_matrix.stack().to_dict()

sl_matrix = df_line_raw.pivot_table(index='From', columns='To', values='Smax')
sl = sl_matrix.stack().to_dict()

In [10]:
# Exporting line set and dictionaries as .pkl 
with open('Data/Parameters/L.pkl', 'wb') as f:
    pickle.dump(L, f)

with open('Data/Parameters/gg.pkl', 'wb') as f:
    pickle.dump(gg, f)
    
with open('Data/Parameters/bb.pkl', 'wb') as f:
    pickle.dump(bb, f)
    
with open('Data/Parameters/sl.pkl', 'wb') as f:
    pickle.dump(sl, f)

## Aggregators

In [11]:
# Synthesizing Aggregator Load Proportions
'''
# This is a multiline comment. Uncomment if necessary
aggr_prop = []
for i in df_load_raw.index:
    temp = []
    for j in range(0, df_load_raw.loc[i, 'Aggr']):
        if j != df_load_raw.loc[i, 'Aggr'] - 1:
            temp.append(np.random.uniform(0.2,0.45))
        else:
            temp.append(1-sum(temp))
    aggr_prop.append(temp)
aggr_prop = [i for aggr in aggr_prop for i in aggr]
df_aggr_raw["Pnom"] = np.ceil(aggr_prop*df_aggr_raw["P"]).astype(int)
df_aggr_raw["Qnom"] = np.ceil(aggr_prop*df_aggr_raw["Q"]).astype(int)
df_aggr_raw.drop(['P', 'Q'], axis=1, inplace=True)
# Setting critical load and normal operation load for aggregators
df_aggr_raw["Pmax"] = np.ceil(1.15*df_aggr_raw["Pnom"]).astype(int)
df_aggr_raw["Pmin"] = np.floor(.7*df_aggr_raw["Pnom"]).astype(int)
df_aggr_raw["Qmax"] = np.ceil(1.15*df_aggr_raw["Qnom"]).astype(int)
df_aggr_raw["Qmin"] = np.floor(0.7*df_aggr_raw["Qnom"]).astype(int)
df_aggr_raw.drop(["Pnom", "Qnom"], axis=1, inplace=True)
df_aggr_raw.to_csv('Data/aggr_raw_limits.csv', index=False)
'''

'\n# This is a multiline comment. Uncomment if necessary\naggr_prop = []\nfor i in df_load_raw.index:\n    temp = []\n    for j in range(0, df_load_raw.loc[i, \'Aggr\']):\n        if j != df_load_raw.loc[i, \'Aggr\'] - 1:\n            temp.append(np.random.uniform(0.2,0.45))\n        else:\n            temp.append(1-sum(temp))\n    aggr_prop.append(temp)\naggr_prop = [i for aggr in aggr_prop for i in aggr]\ndf_aggr_raw["Pnom"] = np.ceil(aggr_prop*df_aggr_raw["P"]).astype(int)\ndf_aggr_raw["Qnom"] = np.ceil(aggr_prop*df_aggr_raw["Q"]).astype(int)\ndf_aggr_raw.drop([\'P\', \'Q\'], axis=1, inplace=True)\n# Setting critical load and normal operation load for aggregators\ndf_aggr_raw["Pmax"] = np.ceil(1.15*df_aggr_raw["Pnom"]).astype(int)\ndf_aggr_raw["Pmin"] = np.floor(.7*df_aggr_raw["Pnom"]).astype(int)\ndf_aggr_raw["Qmax"] = np.ceil(1.15*df_aggr_raw["Qnom"]).astype(int)\ndf_aggr_raw["Qmin"] = np.floor(0.7*df_aggr_raw["Qnom"]).astype(int)\ndf_aggr_raw.drop(["Pnom", "Qnom"], axis=1, inplac

In [12]:
# Synthesizing Socioeconomic Scores of Aggregators
'''
# This is a multiline comment. Uncomment if necessary
df_aggr_raw = pd.read_csv('Data/aggr_raw_limits.csv', index_col=0)
df_aggr_raw
randses = np.random.randint(10, 110, size=len(df_aggr_raw["Mu"]), dtype=int)
randses
df_aggr_raw["Sigma"] = randses
df_aggr_raw.to_csv('Data/ieee24rts_aggregators.csv', index=False)
'''

'\n# This is a multiline comment. Uncomment if necessary\ndf_aggr_raw = pd.read_csv(\'Data/aggr_raw_limits.csv\', index_col=0)\ndf_aggr_raw\nrandses = np.random.randint(10, 110, size=len(df_aggr_raw["Mu"]), dtype=int)\nrandses\ndf_aggr_raw["Sigma"] = randses\ndf_aggr_raw.to_csv(\'Data/ieee24rts_aggregators.csv\', index=False)\n'

In [13]:
# Final processing of aggregator data
df_aggr = pd.read_csv('Data/ieee24rts_aggregators.csv', index_col=False)
df_aggr_old = pd.read_csv('Data/aggr_raw_limits.csv', index_col=False)
df_aggr_final = pd.concat([df_aggr_old['Bus'], df_aggr], axis=1)
df_aggr_final["Gamma"] = np.round(df_aggr_final["Gamma"], decimals=2)
df_aggr_final["Mu"] = np.round(df_aggr_final["Mu"], decimals=4)
df_aggr_final

,Bus,Aggregator,Pmax,Pmin,Qmax,Qmin,Gamma,Mu,Sigma
0,1,1,28,16,6,3,11.05,0.0160,64
1,1,2,98,59,21,12,37.29,0.0590,47
2,2,1,27,16,6,3,10.66,0.0160,57
3,2,2,87,52,19,11,34.33,0.0507,35
4,3,1,59,35,13,7,23.28,0.0347,29
5,3,2,53,32,12,7,20.92,0.0320,75
6,3,3,98,59,21,12,38.67,0.0560,86
7,4,1,28,16,6,3,11.05,0.0160,63
8,4,2,59,35,13,7,23.28,0.0347,107
9,5,1,18,10,4,2,7.10,0.0107,86


In [14]:
# Aggregator Index set
A = set()
for i in df_aggr_final.index:
    agg = (df_aggr_final.loc[i, 'Bus'], df_aggr_final.loc[i,'Aggregator'])
    A.add(agg)

In [15]:
# Generating Dictionaries for Aggregator Parameters
pamax_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Pmax')
p_a_max = pamax_matrix.stack().to_dict()

pamin_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Pmin')
p_a_min = pamin_matrix.stack().to_dict()

qamax_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Qmax')
q_a_max = qamax_matrix.stack().to_dict()

qamin_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Qmin')
q_a_min = qamin_matrix.stack().to_dict()

gamma_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Gamma')
gamma = gamma_matrix.stack().to_dict()

mu_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Mu')
mu = mu_matrix.stack().to_dict()

sigma_matrix = df_aggr_final.pivot_table(index='Bus', columns='Aggregator', values='Sigma')
sigma = sigma_matrix.stack().to_dict()

In [16]:
# Exporting Aggregator Set and Dictionaries as .pkl
with open('Data/Parameters/A.pkl', 'wb') as f:
    pickle.dump(A, f)
    
with open('Data/Parameters/p_a_max.pkl', 'wb') as f:
    pickle.dump(p_a_max, f)
    
with open('Data/Parameters/p_a_min.pkl', 'wb') as f:
    pickle.dump(p_a_min, f)
    
with open('Data/Parameters/q_a_max.pkl', 'wb') as f:
    pickle.dump(q_a_max, f)
    
with open('Data/Parameters/q_a_min.pkl', 'wb') as f:
    pickle.dump(q_a_min, f)
    
with open('Data/Parameters/gamma.pkl', 'wb') as f:
    pickle.dump(gamma, f)
    
with open('Data/Parameters/mu.pkl', 'wb') as f:
    pickle.dump(mu, f)
    
with open('Data/Parameters/sigma.pkl', 'wb') as f:
    pickle.dump(sigma, f)